In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 
import os

In [2]:
# Image data generator
idg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0, validation_split=0.1)

# As flow from directory needs the images to placed in a defined folder structure
# Flow fom dataframe, requires us to create a DataFrame which the image path and its label
df = pd.DataFrame(columns=["image_path","label"])
df["image_path"] = os.listdir("train/")
df.loc[df["image_path"].str.contains("dog"), ["label"]] = "dog"
df.loc[df["image_path"].str.contains("cat"), ["label"]] = "cat"

df.sample(5)

,image_path,label
21733,dog.7058.jpg,dog
21718,dog.7044.jpg,dog
14555,dog.11847.jpg,dog
11006,cat.8654.jpg,cat
5205,cat.3432.jpg,cat


In [3]:
batch_size = 64
train_idg = idg.flow_from_dataframe(dataframe=df, directory="train/", x_col="image_path", y_col="label", 
                                    target_size=(150,150), batch_size=batch_size, subset="training")
val_idg = idg.flow_from_dataframe(dataframe=df, directory="train/", x_col="image_path", y_col="label", 
                                    target_size=(150,150), batch_size=batch_size, subset="validation")

Found 22500 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.


In [6]:
#Build the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input((150,150,3), name="Input"))

#Convolution - Chunk 1 - Con + polling
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), 
                                 padding="valid", activation="relu", name="Conv1"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="Pool1"))

#Convolution - Chunk 2 - Con + polling
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), 
                                 padding="valid", activation="relu", name="Conv2"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="Pool2"))

#Catagorical layers now
#Flattening
model.add(tf.keras.layers.Flatten(name="Flatten"))
#Hidden
model.add(tf.keras.layers.Dense(100, activation="relu", name="Hidden"))
#Output
model.add(tf.keras.layers.Dense(2, activation="softmax", name="Output"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 148, 148, 16)      448       
                                                                 
 Pool1 (MaxPooling2D)        (None, 74, 74, 16)        0         
                                                                 
 Conv2 (Conv2D)              (None, 72, 72, 32)        4640      
                                                                 
 Pool2 (MaxPooling2D)        (None, 36, 36, 32)        0         
                                                                 
 Flatten (Flatten)           (None, 41472)             0         
                                                                 
 Hidden (Dense)              (None, 100)               4147300   
                                                                 
 Output (Dense)              (None, 2)                

In [11]:
# Params for Conv1 - (number of channels * number of filters * number of values in each filter) 
# + number of filters(one bias for each filter)
# 3*16*9 + 16

# Each filter is applied on each channel individually and combined, hence we need to multiply with number of 
# channels to get the total parameters, this is how reduction is happenning on the depth

# No parameters for pooling

# Similarly for Conv2(16 channels)
# 16*9*32 + 32

In [12]:
# Run the model
model.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.categorical_crossentropy, metrics=["acc"])

In [13]:
model_history = model.fit(train_idg, batch_size=batch_size, validation_data=val_idg, epochs=10)

Epoch 1/10
352/352 [==============================] - 185s 524ms/step - loss: 0.6700 - acc: 0.5900 - val_loss: 0.5747 - val_acc: 0.7572
Epoch 2/10
352/352 [==============================] - 180s 511ms/step - loss: 0.6438 - acc: 0.6266 - val_loss: 0.7836 - val_acc: 0.4112
Epoch 3/10
352/352 [==============================] - 181s 514ms/step - loss: 0.6236 - acc: 0.6480 - val_loss: 0.6584 - val_acc: 0.6000
Epoch 4/10
352/352 [==============================] - 181s 513ms/step - loss: 0.6061 - acc: 0.6641 - val_loss: 0.8384 - val_acc: 0.3776
Epoch 5/10
352/352 [==============================] - 180s 512ms/step - loss: 0.5867 - acc: 0.6792 - val_loss: 0.6576 - val_acc: 0.6048
Epoch 6/10
352/352 [==============================] - 181s 514ms/step - loss: 0.5636 - acc: 0.7043 - val_loss: 0.8493 - val_acc: 0.4240
Epoch 7/10
352/352 [==============================] - 181s 514ms/step - loss: 0.5408 - acc: 0.7247 - val_loss: 0.9274 - val_acc: 0.3896
Epoch 8/10
352/352 [============================